<a href="https://colab.research.google.com/github/satyajitghana/TSAI-DeepNLP-END2.0/blob/main/04_RNN_LSTM/04_LSTM_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Tue May 25 18:30:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)

LABEL = data.LabelField(dtype = torch.float)

In [3]:
from torchtext.legacy import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 15.3MB/s]


In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [5]:
print(vars(train_data.examples[0]))

{'text': ['Dressed', 'to', 'Kill', '(', '1980', ')', 'is', 'a', 'mystery', 'horror', 'film', 'from', 'Brian', 'De', 'Palma', 'and', 'it', 'really', 'works', '.', 'The', 'atmosphere', 'is', 'right', 'there', '.', 'The', 'atmosphere', 'that', 'makes', 'you', 'scared', '.', 'And', 'is', "n't", 'that', 'what', 'a', 'horror', 'film', 'is', 'supposed', 'to', 'do', '.', 'All', 'the', 'actors', 'are', 'in', 'the', 'right', 'places', '.', 'Michael', 'Caine', 'is', 'perfect', 'as', 'Dr.', 'Robert', 'Elliott', ',', 'the', 'shrink', 'with', 'a', 'little', 'secret', '.', 'Angie', 'Dickinson', 'as', 'Kate', 'Miller', ',', 'the', 'sexually', 'frustrated', 'mature', 'woman', 'is', 'terrific', '.', 'Keith', 'Gordon', 'as', 'her', 'son', 'Peter', 'is', 'brilliant', '.', 'Nancy', 'Allen', 'as', 'Liz', 'Blake', 'the', 'call', 'girl', 'is', 'fantastic', '.', 'Dennis', 'Franz', 'does', 'his', 'typical', 'detective', 'role', '.', 'His', 'Detective', 'Marino', 'is', 'one', 'of', 'the', 'most', 'colorful', 'in

In [6]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [8]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [10]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 202085), (',', 192344), ('.', 165350), ('and', 109048), ('a', 109006), ('of', 100263), ('to', 93393), ('is', 76161), ('in', 60995), ('I', 54145), ('it', 53577), ('that', 49117), ('"', 44429), ("'s", 43553), ('this', 42185), ('-', 37157), ('/><br', 35889), ('was', 35031), ('as', 30319), ('with', 29691)]


In [11]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [12]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [13]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch=True, # necessary for packed_padded_sequence
    device = device)

In [43]:
print('Train')
for batch in train_iterator:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
print('\nValid:')
for batch in valid_iterator:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
print('\nTest:')
for batch in test_iterator:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Train
Text matrix size: torch.Size([195, 128])
Target vector size: torch.Size([128])

Valid:
Text matrix size: torch.Size([54, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([42, 128])
Target vector size: torch.Size([128])


In [17]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_length):

        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length.to('cpu'))
        
        #[sentence len, batch size, embedding size] => 
        #  output: [sentence len, batch size, hidden size]
        #  hidden: [1, batch size, hidden size]
        packed_output, (hidden, cell) = self.lstm(packed)
        
        return self.fc(hidden.squeeze(0)).view(-1)


In [32]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = LSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [33]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,595,777 trainable parameters


In [44]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=1e-4)
# NOTE: DO NOT USE SGD HERE, ONLY ADAM

In [45]:
criterion = nn.BCEWithLogitsLoss()

In [46]:
model = model.to(device)
criterion = criterion.to(device)

In [47]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [48]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:

        text, text_length = batch.text
        logits = model(text, text_length)
        
        loss = criterion(logits, batch.label)
        
        acc = binary_accuracy(logits, batch.label)
        
        optimizer.zero_grad()

        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [49]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_length = batch.text
            predictions = model(text, text_length)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [50]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [51]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 18s
	Train Loss: 0.688 | Train Acc: 54.21%
	 Val. Loss: 0.686 |  Val. Acc: 54.52%
Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 0.663 | Train Acc: 59.94%
	 Val. Loss: 0.631 |  Val. Acc: 64.53%
Epoch: 03 | Epoch Time: 0m 18s
	Train Loss: 0.579 | Train Acc: 70.00%
	 Val. Loss: 0.553 |  Val. Acc: 72.54%
Epoch: 04 | Epoch Time: 0m 18s
	Train Loss: 0.529 | Train Acc: 74.46%
	 Val. Loss: 0.522 |  Val. Acc: 75.11%
Epoch: 05 | Epoch Time: 0m 18s
	Train Loss: 0.483 | Train Acc: 77.96%
	 Val. Loss: 0.487 |  Val. Acc: 77.80%


In [52]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.491 | Test Acc: 77.13%
